In [5]:
import pandas as pd
import numpy as np

In [2]:
sampled_df = pd.read_csv('C:/Users/D072202/DeepAnyMatch/DeepAnyMatch/result_data/test_2019_03_30_16_08_14_050982/scombined.csv', index_col=['Unnamed: 0'])
stratified_df = pd.read_csv('C:/Users/D072202/DeepAnyMatch/DeepAnyMatch/result_data/test_2019_03_30_16_08_14_050982/strcombined.csv', index_col=['Unnamed: 0'])

new_index = list(range(len(sampled_df)))
sampled_df['index'] = new_index
sampled_df = sampled_df.set_index('index')

new_index = list(range(len(sampled_df), len(sampled_df)+len(stratified_df)))
stratified_df['index'] = new_index
stratified_df = stratified_df.set_index('index')

stratified_df.head()

,src_0,src_1,src_2,src_3,src_4,src_5,src_6,src_7,src_8,src_9,...,tgt_8,tgt_9,label,syntactic_diff,src_tgt_angle,src_angle_to_origin,tgt_angle_to_origin,src_veclen,tgt_veclen,src_tgt_veclen
index,,,,,,,,,,,,,,,,,,,,,
1000,2.339229,-0.392067,0.477628,1.323438,-0.546237,0.580065,0.759125,0.954027,1.243610,-0.470326,...,-0.263621,0.358229,0,0.862069,0.434660,0.529081,0.477593,3.390337,3.390337,3.628273
1001,2.197335,0.208605,1.918295,1.756214,0.152661,1.215641,-0.644312,0.206591,0.962457,-0.619729,...,0.025018,-0.898104,1,0.342857,0.691404,0.542137,0.302365,3.860730,3.860730,2.201456
1002,2.052194,0.488991,1.495510,1.385253,0.533877,1.399131,-0.298412,0.996471,0.706851,-0.817006,...,0.095905,-0.093303,0,0.852941,0.464000,0.606707,0.187116,3.619091,3.619091,3.628775
1003,2.339229,-0.392067,0.477628,1.323438,-0.546237,0.580065,0.759125,0.954027,1.243610,-0.470326,...,0.042120,1.193816,0,0.846154,0.408903,0.529081,0.320651,3.390337,3.390337,3.503827
1004,1.833076,0.007087,1.450763,1.328631,-0.670678,0.695841,-0.566484,1.289831,1.207827,-1.827719,...,0.203093,0.014559,0,0.932515,0.488994,0.370430,0.357976,3.866286,3.866286,3.669463


In [3]:
a = sampled_df.loc[sampled_df['label']==1].sample(n=400, replace=False)
b = sampled_df.loc[sampled_df['label']==0].sample(n=100, replace=False)
c = stratified_df.loc[stratified_df['label']==1].sample(n=0, replace=False)
d = stratified_df.loc[stratified_df['label']==0].sample(n=400, replace=False)
train = d.append(c.append(a.append(b, ignore_index=False), ignore_index=False), ignore_index=False)

In [4]:
a = sampled_df.loc[sampled_df['label']==1].sample(n=400, replace=False)
b = sampled_df.loc[sampled_df['label']==0].sample(n=100, replace=False)
c = stratified_df.loc[stratified_df['label']==1].sample(n=0, replace=False)
d = stratified_df.loc[stratified_df['label']==0].sample(n=600, replace=False)
train2 = d.append(c.append(a.append(b, ignore_index=False), ignore_index=False), ignore_index=False)

In [283]:
x_train = train.loc[:, train.columns != 'label']
y_train = train['label']
x_train2 = train2.loc[:, train2.columns != 'label']
y_train2 = train2['label']

test = stratified_df.loc[(~stratified_df.index.isin(train.index)) & (~stratified_df.index.isin(train2.index))]
x_test = test.loc[:, test.columns != 'label']
y_test = test['label']

In [284]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

In [285]:
model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=0)
model = model.fit(x_train, y_train)
prediction = model.predict(x_test)
result = classification_report(prediction, np.array(y_test), target_names=['true','false'])
print(result)

              precision    recall  f1-score   support

        true       0.88      0.98      0.93       195
       false       0.93      0.58      0.72        65

   micro avg       0.88      0.88      0.88       260
   macro avg       0.90      0.78      0.82       260
weighted avg       0.89      0.88      0.87       260



In [286]:
model2 = LogisticRegression(solver='lbfgs')
model2 = model2.fit(pd.DataFrame(x_train['syntactic_diff']), y_train)
prediction2 = model2.predict(pd.DataFrame(x_test['syntactic_diff']))
result2 = classification_report(prediction2, np.array(y_test), target_names=['true','false'])
print(result2)

              precision    recall  f1-score   support

        true       0.89      0.93      0.91       209
       false       0.66      0.53      0.59        51

   micro avg       0.85      0.85      0.85       260
   macro avg       0.77      0.73      0.75       260
weighted avg       0.84      0.85      0.85       260



In [288]:
prediction3 = list()
for i in range(len(prediction)):
    if prediction[i] == 1 and prediction2[i] == 1:
        prediction3.append(1)
    else:
        prediction3.append(0)
prediction3 = np.array(prediction3)
result3 = classification_report(prediction3, np.array(y_test), target_names=['true','false'])
print(result3) 

              precision    recall  f1-score   support

        true       0.92      0.94      0.93       216
       false       0.66      0.61      0.64        44

   micro avg       0.88      0.88      0.88       260
   macro avg       0.79      0.77      0.78       260
weighted avg       0.88      0.88      0.88       260

